# Implementación de algoritmo de detección de Uckert, Bhartia y Michel

Basado en el trabajo https://doi.org/10.1177/0003702819850584

Carga de librerías externas, configuración de los parámetros a ser explorados durante la medición y nombre del archivo de donde se cargarán los datos espectrales.

In [ ]:
import math
import copy
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt
from random import random, sample
from joblib import Parallel, delayed
from scipy.stats import norm, uniform, poisson, iqr
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

archivo_carga_espectros = '../resultados.ndjson'

params = [
    {
        'sigma_clip': 3,
        'k': 8
    },
    {
        'sigma_clip': 3,
        'k': 9
    },
    {
        'sigma_clip': 3,
        'k': 10
    },
    {
        'sigma_clip': 3,
        'k': 11,
    },
    {
        'sigma_clip': 3,
        'k': 12
    },
    {
        'sigma_clip': 4,
        'k': 8
    },
    {
        'sigma_clip': 4,
        'k': 9
    },
    {
        'sigma_clip': 4,
        'k': 10
    },
    {
        'sigma_clip': 4,
        'k': 11,
    },
    {
        'sigma_clip': 4,
        'k': 12
    },
    {
        'sigma_clip': 5,
        'k': 8
    },
    {
        'sigma_clip': 5,
        'k': 9
    },
    {
        'sigma_clip': 5,
        'k': 10
    },
    {
        'sigma_clip': 5,
        'k': 11,
    },
    {
        'sigma_clip': 5,
        'k': 12
    },
]

# Posición más alta del vector que actúa como eje X
max_x = 1024

# Distribución de probabilidad a trozos que define la anchura de los spikes
# las constantes se escogen de forma que la probabilidad de un spike esté conformado por:
# - 5 puntos espectrales sea del 1%
# - 4 puntos espectrales sea del 4%
# - 3 puntos espectrales sea del 10%
# - 2 puntos espectrales sea del 15%
# - 1 punto espectral sea del 70%
def generador_anchura_picos():
    var = random()

    if var < .01:
        return (max_x / 1024) * 5 / 10
    elif var < .05:
        return (max_x / 1024) * 4 / 10
    elif var < .15:
        return (max_x / 1024) * 3 / 10
    elif var < .3:
        return (max_x / 1024) * 2 / 10
    else:
        return (max_x / 1024) * 1 / 10

config = {
    'dist_num_spikes': uniform(loc=0, scale=8),
    'dist_posicion_spikes': uniform(loc=1, scale=max_x - 1),
    'dist_altura_spikes': norm(loc=35, scale=30),
    'dist_anchura_spikes': generador_anchura_picos,
    'dist_ruido_porcentual_inyectado': norm(loc=0, scale=0.02),
    'numero_procesos_concurrentes': 10,
    'factor_deteccion_spikes': 0.01
}

config_uckert_bhartia_ramser = {
    'tamano_conjunto_evaluacion': 30
}

Funciones de carga de espectros

In [ ]:
def deserializar_espectro(espectro):
    espectro['vector_base'] = np.array(espectro['vector_base'])
    espectro['espectro_base_muestra'] = np.array(espectro['espectro_base_muestra'])
    espectro['espectro_base_background'] = np.array(espectro['espectro_base_background'])
    espectro['baseline_muestra'] = np.array(espectro['baseline_muestra'])
    espectro['baseline_background'] = np.array(espectro['baseline_background'])
    espectro['muestra_con_baseline'] = np.array(espectro['muestra_con_baseline'])
    espectro['background_con_baseline'] = np.array(espectro['background_con_baseline'])
    espectro['muestra_combinado_base'] = np.array(espectro['muestra_combinado_base'])
    espectro['espectro_ruido_combinado'] = np.array(espectro['espectro_ruido_combinado'])
    espectro['espectro_ruido_background'] = np.array(espectro['espectro_ruido_background'])
    espectro['spikes_muestra'] = np.array(espectro['spikes_muestra'])
    espectro['spikes_background'] = np.array(espectro['spikes_background'])
    espectro['flag_spikes_muestra'] = np.array(espectro['flag_spikes_muestra'])
    espectro['flag_spikes_background'] = np.array(espectro['flag_spikes_background'])
    espectro['muestra_base_con_spikes'] = np.array(espectro['muestra_base_con_spikes'])
    espectro['y_muestra'] = np.array(espectro['y_muestra'])
    espectro['y_background'] = np.array(espectro['y_background'])

    return espectro

def cargar_espectros():
    with open(archivo_carga_espectros, 'r') as fp:
        data = []
        for line in fp:
            data.append(deserializar_espectro(json.loads(line)))
        fp.close()
        return data

espectros = cargar_espectros()

Función para calcular métricas clasificación de spikes.

In [ ]:
def calcular_metricas_clasificacion(valores_reales, predicciones):
    _predicciones = np.array(predicciones).flatten()
    _valores_reales = np.array(valores_reales).flatten()

    return precision_recall_fscore_support(_valores_reales, _predicciones), confusion_matrix(_valores_reales, _predicciones)

In [ ]:
# El factor de anchura indica que aproximadamente a 1.4 veces su valor, a cada lado del centro,
# la distribución alcanza un 1% de su valor máximo
def generar_perfil_normal(vector_base, maximo, posicion, factor_anchura):
    argumento = (vector_base - posicion) / factor_anchura
    return maximo * np.exp(-4 * math.log(2) * (argumento ** 2))

# El factor de anchura indica que aproximadamente a 5 veces su valor, a cada lado del centro,
# la distribución alcanza un 1% de su valor máximo
def generar_perfil_cauchy(vector_base, maximo, posicion, factor_anchura):
    argumento = (vector_base - posicion) / factor_anchura
    return maximo / (1 + (argumento ** 2))

# El factor de anchura indica que aproximadamente a 4.5 a 4.9 veces su valor, a cada lado del centro,
# la distribución alcanza un 1% de su valor máximo
def generar_perfil_voigt(vector_base, maximo, posicion, factor_anchura):
    posicion_normal_cauchy = (posicion + len(vector_base) / 2) / 2
    perfil_normal = generar_perfil_normal(
        vector_base,
        maximo,
        posicion_normal_cauchy,
        factor_anchura
    )
    perfil_cauchy = generar_perfil_cauchy(
        vector_base,
        maximo,
        posicion_normal_cauchy,
        factor_anchura
    )

    perfil_voigt = np.convolve(perfil_normal, perfil_cauchy, 'same')

    maximo_perfil_voigt = np.max(perfil_voigt)

    if maximo_perfil_voigt == 0:
        return perfil_voigt

    return perfil_voigt * maximo / maximo_perfil_voigt

Funciones de reconstrucción de picos Raman a partir de parámetros de generación.

In [ ]:
def generar_picos_raman(vector_base, lista_parametros):
    if len(lista_parametros) == 0:
        return np.zeros(len(vector_base))

    perfiles_voigt = [
        generar_perfil_voigt(
            vector_base,
            altura,
            posicion,
            anchura
        ) for (altura, posicion, anchura) in lista_parametros
    ]

    return np.sum(perfiles_voigt, axis=0)

Funciones de reconstrucción de polinomios baseline a partir de parámetros de generación.

In [ ]:
umbral_diferencia_raices = 0.3

def generar_polinomio_baseline(vector_base, optimos, factor_lineal_especial, factor_altura, debe_voltearse, sumando_altura):
    coefs_derivada = []

    if len(optimos) > 0:
        coefs_derivada = np.polynomial.polynomial.polyfromroots(optimos)
    else:
        coefs_derivada = [factor_lineal_especial]

    if debe_voltearse:
        coefs_derivada = [-1 * coef for coef in coefs_derivada]

    coefs_pol = [coef / (n + 1) for n, coef in enumerate(coefs_derivada)]
    coefs_pol.insert(0, 0)

    lim_rango_inferior = -1
    lim_rango_superior = 1
    if len(optimos) > 0:
        lim_rango_inferior = np.min(optimos) - umbral_diferencia_raices
        lim_rango_superior = np.max(optimos) + umbral_diferencia_raices

    rango = np.linspace(lim_rango_inferior, lim_rango_superior, len(vector_base))
    pol = np.polyval(np.flip(coefs_pol), rango)

    pol_escalado = pol * factor_altura / np.max(np.abs(pol))

    polinomio_final = abs(np.min(pol_escalado)) + sumando_altura * factor_altura + pol_escalado

    return polinomio_final

Funciones de generación de spikes

In [ ]:
def generar_spikes(vector_base, lista_parametros):
    flag_spikes = np.full(len(vector_base), False)

    if len(lista_parametros) == 0:
        return np.zeros(len(vector_base)), flag_spikes

    perfiles_cauchy = [
        generar_perfil_cauchy(
            vector_base,
            altura,
            posicion,
            anchura
        ) for (altura, posicion, anchura) in lista_parametros
    ]

    for spike in perfiles_cauchy:
        flag_spikes = flag_spikes | (spike > config['factor_deteccion_spikes'] * np.max(spike))

    return np.sum(perfiles_cauchy, axis=0), flag_spikes

def generar_spikes_aleatorios(vector_base, dist_num_spikes, dist_posicion_spikes, dist_altura_spikes, dist_anchura_spikes):
    num_spikes = round(dist_num_spikes.rvs())

    lista_parametros = []

    for _ in range(num_spikes):
        altura = abs(dist_altura_spikes.rvs())
        posicion = round(dist_posicion_spikes.rvs())
        anchura = abs(dist_anchura_spikes())

        lista_parametros.append((altura, posicion, anchura))

    spikes, flag_spikes = generar_spikes(vector_base, lista_parametros)

    return spikes, lista_parametros, flag_spikes

In [ ]:
def anadir_ruido_poisson_espectro(espectro, **poisson_params):
    maximo_espectro = np.max(espectro)
    k_poisson = None
    dist_param_poisson = None

    for key, value in poisson_params.items():
        if key == 'k_poisson':
            k_poisson = value
        elif key == 'dist_param_poisson':
            dist_param_poisson = value

    if dist_param_poisson != None:
        k_poisson = dist_param_poisson.rvs()

    espectro_ruido = poisson.rvs(k_poisson * espectro)
    maximo_espectro_ruido = np.max(espectro_ruido)
    if maximo_espectro_ruido > 0:
        espectro_ruido = espectro_ruido * maximo_espectro / maximo_espectro_ruido

    return espectro_ruido

Funciones para generar espectros correlacionados a un espectro base.

In [ ]:
def inyectar_ruido_parametro(parametro, dist_ruido):
    return parametro * (1 + dist_ruido.rvs())

def paso_5_aleatorio(vector_base, dist_num_spikes, dist_posicion_spikes, dist_altura_spikes, dist_anchura_spikes):
    spikes_muestra, parametros_spike_muestra, flag_spikes_muestra = generar_spikes_aleatorios(
        vector_base,
        dist_num_spikes,
        dist_posicion_spikes,
        dist_altura_spikes,
        dist_anchura_spikes
    )

    spikes_background, parametros_spike_background, flag_spikes_background = generar_spikes_aleatorios(
        vector_base,
        dist_num_spikes,
        dist_posicion_spikes,
        dist_altura_spikes,
        dist_anchura_spikes
    )

    return spikes_muestra, parametros_spike_muestra, flag_spikes_muestra, spikes_background, parametros_spike_background, flag_spikes_background

def generar_espectro_correlacionado(config, datos_espectro_base):
    datos = {}
    copia_espectro = copy.deepcopy(datos_espectro_base)
    vector_base = copia_espectro['vector_base']

    datos['vector_base'] = vector_base

    y_muestra = np.zeros(len(vector_base))
    y_background = np.zeros(len(vector_base))

    dist_ruido_inyectado = config['dist_ruido_porcentual_inyectado']

    parametros_espectro_base_muestra = copia_espectro['parametros_espectro_base_muestra']
    parametros_espectro_base_background = copia_espectro['parametros_espectro_base_background']

    parametros_espectro_base_muestra_ruido = [
        (
            abs(inyectar_ruido_parametro(altura, dist_ruido_inyectado)),
            round(inyectar_ruido_parametro(posicion, dist_ruido_inyectado)),
            abs(inyectar_ruido_parametro(anchura, dist_ruido_inyectado))
        ) for (altura, posicion, anchura) in parametros_espectro_base_muestra
    ]

    parametros_espectro_base_background = [
        (
            abs(inyectar_ruido_parametro(altura, dist_ruido_inyectado)),
            round(inyectar_ruido_parametro(posicion, dist_ruido_inyectado)),
            abs(inyectar_ruido_parametro(anchura, dist_ruido_inyectado))
        ) for (altura, posicion, anchura) in parametros_espectro_base_background
    ]

    espectro_muestra = generar_picos_raman(vector_base, parametros_espectro_base_muestra_ruido)
    espectro_background = generar_picos_raman(vector_base, parametros_espectro_base_background)

    datos['espectro_base_muestra'] = espectro_muestra
    datos['espectro_base_background'] = espectro_background
    datos['parametros_espectro_base_muestra'] = parametros_espectro_base_muestra_ruido
    datos['parametros_espectro_base_background'] = parametros_espectro_base_background
    y_muestra = y_muestra + espectro_muestra
    y_background = y_background + espectro_background
    datos['muestra_con_baseline'] = y_muestra
    datos['background_con_baseline'] = y_background

    nuevos_optimos_baseline_muestra = [
        inyectar_ruido_parametro(optimo, dist_ruido_inyectado) for optimo in copia_espectro['parametros_baseline_muestra']['optimos']
    ]
    nuevo_factor_altura_baseline_muestra = abs(inyectar_ruido_parametro(copia_espectro['parametros_baseline_muestra']['factor_altura'], dist_ruido_inyectado))
    nuevo_sumando_altura_muestra = abs(inyectar_ruido_parametro(copia_espectro['parametros_baseline_muestra']['sumando_altura'], dist_ruido_inyectado))

    nuevos_optimos_baseline_background = [
        inyectar_ruido_parametro(optimo, dist_ruido_inyectado) for optimo in copia_espectro['parametros_baseline_background']['optimos']
    ]
    nuevo_factor_altura_baseline_background = inyectar_ruido_parametro(copia_espectro['parametros_baseline_background']['factor_altura'], dist_ruido_inyectado)
    nuevo_sumando_altura_background = inyectar_ruido_parametro(copia_espectro['parametros_baseline_background']['sumando_altura'], dist_ruido_inyectado)

    baseline_muestra = generar_polinomio_baseline(
        vector_base,
        nuevos_optimos_baseline_muestra,
        copia_espectro['parametros_baseline_muestra']['factor_lineal_especial'],
        nuevo_factor_altura_baseline_muestra,
        copia_espectro['parametros_baseline_muestra']['debe_voltearse'],
        nuevo_sumando_altura_muestra
    )

    parametros_baseline_muestra = {
        'optimos': nuevos_optimos_baseline_muestra,
        'factor_lineal_especial': copia_espectro['parametros_baseline_muestra']['factor_lineal_especial'],
        'factor_altura': nuevo_factor_altura_baseline_muestra,
        'sumando_altura': nuevo_sumando_altura_muestra,
        'debe_voltearse': copia_espectro['parametros_baseline_muestra']['debe_voltearse']
    }

    baseline_background = generar_polinomio_baseline(
        vector_base,
        nuevos_optimos_baseline_background,
        copia_espectro['parametros_baseline_background']['factor_lineal_especial'],
        nuevo_factor_altura_baseline_background,
        copia_espectro['parametros_baseline_background']['debe_voltearse'],
        nuevo_sumando_altura_background
    )

    parametros_baseline_background = {
        'optimos': nuevos_optimos_baseline_background,
        'factor_lineal_especial': copia_espectro['parametros_baseline_background']['factor_lineal_especial'],
        'factor_altura': nuevo_factor_altura_baseline_background,
        'sumando_altura': nuevo_sumando_altura_background,
        'debe_voltearse': copia_espectro['parametros_baseline_background']['debe_voltearse']
    }

    datos['baseline_muestra'] = baseline_muestra
    datos['baseline_background'] = baseline_background
    datos['parametros_baseline_muestra'] = parametros_baseline_muestra
    datos['parametros_baseline_background'] = parametros_baseline_background

    y_muestra = y_muestra + baseline_muestra
    y_background = y_background + baseline_background

    factor_comb_lineal = copia_espectro['factor_comb_lineal']

    espectro_combinado = factor_comb_lineal * y_muestra + (1 - factor_comb_lineal) * y_background

    datos['factor_comb_lineal'] = factor_comb_lineal
    datos['muestra_combinado_base'] = espectro_combinado

    k_combinado_poisson = copia_espectro['k_combinado_poisson']
    k_background_poisson = copia_espectro['k_background_poisson']

    espectro_ruido_combinado = poisson.rvs(k_combinado_poisson * espectro_combinado)
    espectro_ruido_background = poisson.rvs(k_background_poisson * y_background)

    maximo_espectro_ruido_combinado = np.max(espectro_ruido_combinado)

    if maximo_espectro_ruido_combinado > 0:
        espectro_ruido_combinado = espectro_ruido_combinado * np.max(espectro_combinado) / maximo_espectro_ruido_combinado

    maximo_espectro_ruido_background = np.max(espectro_ruido_background)

    if maximo_espectro_ruido_background > 0:
        espectro_ruido_background = espectro_ruido_background * np.max(y_background) / maximo_espectro_ruido_background

    datos['k_combinado_poisson'] = k_combinado_poisson
    datos['k_background_poisson'] = k_background_poisson
    datos['espectro_ruido_combinado'] = espectro_ruido_combinado
    datos['espectro_ruido_background'] = espectro_ruido_background

    spikes_muestra, parametros_spike_muestra, flag_spikes_muestra, spikes_background, parametros_spike_background, flag_spikes_background = paso_5_aleatorio(
        vector_base,
        config['dist_num_spikes'],
        config['dist_posicion_spikes'],
        config['dist_altura_spikes'],
        config['dist_anchura_spikes']
    )

    y_muestra = espectro_ruido_combinado + spikes_muestra
    y_background = espectro_ruido_background + spikes_background

    datos['spikes_muestra'] = spikes_muestra
    datos['spikes_background'] = spikes_background
    datos['parametros_spike_muestra'] = parametros_spike_muestra
    datos['parametros_spike_background'] = parametros_spike_background
    datos['flag_spikes_muestra'] = flag_spikes_muestra
    datos['flag_spikes_background'] = flag_spikes_background

    muestra_base_con_spikes = espectro_muestra + spikes_muestra

    muestra_base_con_spikes_ruido = anadir_ruido_poisson_espectro(muestra_base_con_spikes, k_poisson=k_combinado_poisson)

    datos['muestra_base_con_spikes'] = muestra_base_con_spikes_ruido

    datos['y_muestra'] = y_muestra
    datos['y_background'] = y_background

    return datos

Algoritmo Uckert-Bhartia-Michel

In [ ]:
def condicion_dos(valor, histograma):
    # Calculo el índice del bin que contiene el valor
    indice = len(histograma[1] >= valor) - 1

    # La condición se cumple si hay algún bin con frecuencia 0
    # a la izquierda del bin que contiene al valor
    if indice >= 0:
        return np.any(histograma[0][:indice] == 0)

    # De lo contrario retorna False
    return False

def algoritmo_uckert_bhartia_michel(input, sigma_clip = 4, k = 10):
    # Puesto que es necesario transponer el array para ejectuar el algoritmo,
    # primero realizo una copia para no manipular el original
    input_copy = np.array(input, dtype='object')

    # Al transponer el array, tengo una lista por cada banda (longitud de onda)
    # de los espectros del input
    mediciones_por_bandas = np.transpose(input_copy)

    # Se calcula la media y desviación estándar de cada banda
    medias = [np.mean(columnas) for columnas in mediciones_por_bandas]
    desviaciones_estandar = [np.std(columnas) for columnas in mediciones_por_bandas]

    # Se debe realizar un corte inicial de los outliers en cada banda, previo a calcular un umbral por banda
    mascaras_corte_inicial = [
      (medias[i] - (sigma_clip * desviaciones_estandar[i]) < banda) & (banda < medias[i] + (sigma_clip * desviaciones_estandar[i])) for i, banda in enumerate(mediciones_por_bandas)
    ]
    # Se construyen las bandas libres de outliers, a ser empleadas para calcular umbrales
    bandas_clipped = [banda[mascaras_corte_inicial[i]] for i, banda in enumerate(mediciones_por_bandas)]
    umbrales = [np.mean(banda) + (k * np.std(banda)) if len(banda) > 0 else 0 for banda in bandas_clipped]

    # El siguiente paso es calcular histogramas por cada banda, el tamano del bin
    # debe seguir la regla de Freedman-Draconis
    tamanos_bin = [2 * iqr(banda) / np.cbrt(len(banda)) for banda in mediciones_por_bandas]

    detecciones = []
    # For loop por legibilidad
    for i, banda in enumerate(mediciones_por_bandas):
        # Construcción del histograma
        minimo = np.min(banda)
        maximo = np.max(banda)
        rango = maximo - minimo
        if tamanos_bin[i] == 0:
            tamanos_bin[i] = 3

        num_bins = math.ceil(rango / tamanos_bin[i])

        histograma = np.histogram(banda, bins=np.linspace(minimo, maximo, num_bins))

        # Se considera una detección cuando el valor de intensidad en una banda supera el umbral
        # y adicionalmente el bin que contiene ese valor de intensidad en el histograma
        # tiene al menos un bin vacío a su izquierda
        detecciones_banda = [umbrales[i] < abs(valor) and condicion_dos(valor, histograma) for valor in banda]

        detecciones.append(detecciones_banda)

    return np.transpose(detecciones)

def aux_paralelizacion_uckert_bhartia_michel(espectro, tamano_conjunto_evaluacion, sigma_clip, k, ruido):
    return algoritmo_uckert_bhartia_michel(
        preprocesado_espectros_uckert_bhartia_michel(espectro, tamano_conjunto_evaluacion, ruido),
        sigma_clip,
        k
    )[0]

def obtener_resultado_parametro_uckert_bhartia_michel(espectros, etiquetas, tamano_conjunto_evaluacion, sigma_clip = 4, k = 10, ruido = False):
    predicciones_uckert_bhartia_michel = Parallel(n_jobs=config['numero_procesos_concurrentes'])(
        delayed(aux_paralelizacion_uckert_bhartia_michel)(espectro, tamano_conjunto_evaluacion, sigma_clip, k, ruido) for espectro in espectros
    )

    return predicciones_uckert_bhartia_michel, calcular_metricas_clasificacion(etiquetas, predicciones_uckert_bhartia_michel)

def visualizar_resultado_uckert_bhartia_michel(vector_base, espectro, etiquetas, tamano_conjunto_evaluacion, sigma_clip = 4, k = 10, ruido = False):
    input = espectro['muestra_base_con_spikes'] if ruido else espectro['espectro_base_muestra'] + espectro['spikes_muestra']

    detecciones = algoritmo_uckert_bhartia_michel(
        preprocesado_espectros_uckert_bhartia_michel(espectro, tamano_conjunto_evaluacion, ruido),
        sigma_clip,
        k
    )[0]

    print(np.any(detecciones))

    fig = plt.figure(figsize=[45, 15], constrained_layout=True)

    fig.suptitle("Muestra de resultados al aplicar el algoritmo Uckert-Bhartia-Michel", fontsize=24, fontweight='bold')

    subfigs = fig.subfigures(nrows=2, ncols=1)

    for row, subfig in enumerate(subfigs):
        (ax) = subfig.subplots(nrows=1, ncols=1)

        if row == 0:
            ax.set_title("Serie original", fontsize=18)
            ax.plot(vector_base, input)
        elif row == 1:
            categorias = {
                'tp': {
                    'color': 'blue',
                    'label': 'Detecciones'
                },
                'fp': {
                    'color': 'black',
                    'label': 'Falsos positivos'
                },
                'fn': {
                    'color': 'red',
                    'label': 'Falsos negativos'
                }
            }

            detecciones_agrupadas = []

            for i, es_resultado in enumerate(detecciones):
                label = ''
                es_spike = etiquetas[i]

                if es_spike:
                    if es_resultado:
                        label = 'tp'
                    else:
                        label = 'fn'
                elif es_resultado:
                    label = 'fp'
                else:
                    label = 'tn'
                detecciones_agrupadas.append(label)

            detecciones_agrupadas = np.array(detecciones_agrupadas)

            ax.set_title("Serie con spikes detectados y tipos de detección", fontsize=18)
            ax.plot(vector_base, input)
            ax.scatter(vector_base[detecciones_agrupadas == 'tp'], input[detecciones_agrupadas == 'tp'], c=categorias['tp']['color'], label=categorias['tp']['label'], s=100)
            ax.scatter(vector_base[detecciones_agrupadas == 'fp'], input[detecciones_agrupadas == 'fp'], c=categorias['fp']['color'], label=categorias['fp']['label'], s=100)
            ax.scatter(vector_base[detecciones_agrupadas == 'fn'], input[detecciones_agrupadas == 'fn'], c=categorias['fn']['color'], label=categorias['fn']['label'], s=100)

            ax.legend(fontsize=28)

def aux_preprocesamiento_seleccion_ruido(config, datos_espectrales, ruido):
    espectro_correlacionado = generar_espectro_correlacionado(config, datos_espectrales)

    if ruido:
        return espectro_correlacionado['muestra_base_con_spikes']
    else:
        return espectro_correlacionado['espectro_base_muestra'] + espectro_correlacionado['spikes_muestra']

def aux_seleccion_espectro(datos_espectrales, ruido):
    if ruido:
        return datos_espectrales['muestra_base_con_spikes']
    else:
        return datos_espectrales['espectro_base_muestra'] + datos_espectrales['spikes_muestra']

def preprocesado_espectros_uckert_bhartia_michel(datos_espectrales, tamano_conjunto_evaluacion, ruido):
    return [
        aux_seleccion_espectro(datos_espectrales, ruido)
        if i == 0 
        else aux_preprocesamiento_seleccion_ruido(
            config,
            datos_espectrales,
            ruido
        ) for i in range(tamano_conjunto_evaluacion)
    ]

Separación de las etiquetas que marcan la posición real de los spikes añadidos del conjunto de datos espectrales cargado

In [ ]:
etiquetas = [espectro['flag_spikes_muestra'] for espectro in espectros]

Recogida de datos y guardado de los resultados en un archivo.

In [ ]:
resultados_uckert_bhartia_michel = []
tamano_conjunto_evaluacion = config_uckert_bhartia_ramser['tamano_conjunto_evaluacion']

for param in params:
    predicciones_uckert_bhartia_michel_ruido, metricas_uckert_bhartia_michel_ruido = obtener_resultado_parametro_uckert_bhartia_michel(
        espectros,
        etiquetas,
        tamano_conjunto_evaluacion,
        param['sigma_clip'],
        param['k'],
        ruido=True
    )
    predicciones_uckert_bhartia_michel_sin_ruido, metricas_uckert_bhartia_michel_sin_ruido = obtener_resultado_parametro_uckert_bhartia_michel(
        espectros,
        etiquetas,
        tamano_conjunto_evaluacion,
        param['sigma_clip'],
        param['k'],
        ruido=False
    )

    for i in range(len(predicciones_uckert_bhartia_michel_ruido)):
        resultados_uckert_bhartia_michel.append({
            'etiquetas': etiquetas[i],
            'sigma_clip': param['sigma_clip'],
            'k': param['k'],
            'predicciones_ruido': predicciones_uckert_bhartia_michel_ruido[i],
            'predicciones_sin_ruido': predicciones_uckert_bhartia_michel_sin_ruido[i],
            'espectro_ruido': espectros[i]['muestra_base_con_spikes'],
            'espectro_sin_ruido': espectros[i]['espectro_base_muestra'] + espectros[i]['spikes_muestra']
        })

with open('resultados_uckert_bhartia_michel', 'wb') as fp:
    pickle.dump(resultados_uckert_bhartia_michel, fp)

Toma de una muestra aleatoria para control de calidad

In [ ]:
indice_muestra = sample(list(range(len(espectros))), 1)[0]

Visualización del resultado de la aplicación del algoritmo Uckert-Bhartia-Michel a un espectro aleatorio al que no le fue añadido ruido aleatorio

In [ ]:
visualizar_resultado_uckert_bhartia_michel(
    espectros[indice_muestra]['vector_base'],
    espectros[indice_muestra],
    etiquetas[indice_muestra],
    tamano_conjunto_evaluacion,
    ruido=False
)

Visualización del resultado de la aplicación del algoritmo Uckert-Bhartia-Michel a un espectro aleatorio al que le fue añadido ruido aleatorio

In [ ]:
visualizar_resultado_uckert_bhartia_michel(
    espectros[indice_muestra]['vector_base'],
    espectros[indice_muestra],
    etiquetas[indice_muestra],
    tamano_conjunto_evaluacion,
    ruido=True
)